In [159]:
import pandas as pd

traindf = pd.read_csv('data/train.csv')
testdf = pd.read_csv('data/test.csv')
traindf['Set'] = 'Train'
testdf['Set'] = 'Test'
togeth = pd.concat([testdf,traindf])
homeplanets = ['Earth', 'Europa', 'Mars']

In [160]:
split_df = togeth['PassengerId'].str.split('_', expand=True)
togeth[['Group', 'GroupNumber']] = split_df

split_df =  togeth['Cabin'].str.split("/", expand = True)
togeth[['Deck', 'Number', 'Side']]= split_df


## datasplits 

In [161]:
def splits(dataframe):
    split_df = dataframe['PassengerId'].str.split('_', expand=True)
    dataframe[['Group', 'GroupNumber']] = split_df

    split_df =  dataframe['Cabin'].str.split("/", expand = True)
    dataframe[['Deck', 'Number', 'Side']]= split_df
    
    split_df = dataframe['Name'].str.split(" ",expand = True)
    dataframe[['FristName','LastName']] = split_df


    return dataframe

togeth = splits(togeth)


In [162]:
togeth

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,Set,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,...,Nelly Carsoning,Test,NaN,0013,01,G,3,S,Nelly,Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,...,Lerome Peckers,Test,NaN,0018,01,F,4,S,Lerome,Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,...,Sabih Unhearfus,Test,NaN,0019,01,C,0,S,Sabih,Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,...,Meratz Caltilter,Test,NaN,0021,01,C,1,S,Meratz,Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,...,Brence Harperez,Test,NaN,0023,01,F,5,S,Brence,Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,...,Gravior Noxnuther,Train,False,9276,01,A,98,P,Gravior,Noxnuther
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,...,Kurta Mondalley,Train,False,9278,01,G,1499,S,Kurta,Mondalley
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,...,Fayey Connon,Train,True,9279,01,G,1500,S,Fayey,Connon
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,...,Celeon Hontichre,Train,False,9280,01,E,608,S,Celeon,Hontichre


## Certain imputes

#### Same name implies home planet

write how many there are beforehand with same last name and therefore home planet 

In [163]:
togeth_cleaned = togeth.dropna(subset=['HomePlanet','LastName'])

# Group by 'LastName' and aggregate HomePlanet into lists
grouped = togeth_cleaned.groupby('LastName')['HomePlanet'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(homeplanets):
    return len(set(homeplanets)) == 1

# Apply the function to identify uniform and mixed groups
grouped['UniformPlanet'] = grouped['HomePlanet'].apply(is_uniform)

# Count how many people are in uniform and mixed planet groups
uniform_groups = grouped[grouped['UniformPlanet'] == True]
mixed_groups = grouped[grouped['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed groups
people_uniform = uniform_groups['HomePlanet'].apply(len).sum()
people_mixed = mixed_groups['HomePlanet'].apply(len).sum()

print(f"Number of people from groups where all share the same HomePlanet: {people_uniform}")
print(f"Number of people from groups where not all share the same HomePlanet: {people_mixed}")


Number of people from groups where all share the same HomePlanet: 12398
Number of people from groups where not all share the same HomePlanet: 0


#### same group implies homeplanet 

In [164]:
togeth_cleaned = togeth.dropna(subset=['HomePlanet','Group'])


# Group by 'LastName' and aggregate HomePlanet into lists
grouped = togeth_cleaned.groupby('Group')['HomePlanet'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(homeplanets):
    return len(set(homeplanets)) == 1

# Apply the function to identify uniform and mixed groups
grouped['UniformPlanet'] = grouped['HomePlanet'].apply(is_uniform)

# Count how many people are in uniform and mixed planet groups
uniform_groups = grouped[grouped['UniformPlanet'] == True]
mixed_groups = grouped[grouped['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed groups
people_uniform = uniform_groups['HomePlanet'].apply(len).sum()
people_mixed = mixed_groups['HomePlanet'].apply(len).sum()

print(f"Number of people from groups where all share the same HomePlanet: {people_uniform}")
print(f"Number of people from groups where not all share the same HomePlanet: {people_mixed}")


Number of people from groups where all share the same HomePlanet: 12682
Number of people from groups where not all share the same HomePlanet: 0


#### impute findings

In [165]:
togeth[togeth.Group == '0753']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,Set,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName
711,0753_01,Europa,True,A/4/P,NaN,34.0,False,0.0,0.0,0.0,...,Mesarix Ennalve,Train,True,0753,01,A,4,P,Mesarix,Ennalve
712,0753_02,Europa,False,A/4/P,55 Cancri e,65.0,False,19.0,1344.0,4.0,...,NaN,Train,False,0753,02,A,4,P,NaN,NaN
713,0753_03,Europa,True,NaN,55 Cancri e,42.0,False,0.0,0.0,0.0,...,Matan Ennalve,Train,True,0753,03,NaN,NaN,NaN,Matan,Ennalve


In [169]:
checks(togeth)


Earth
Deck
G    3731
F    2437
E     589
Name: count, dtype: int64

Europa
Deck
B    1137
C    1096
A     352
D     301
E     199
T      10
Name: count, dtype: int64

Mars
Deck
F    1742
E     515
D     410
Name: count, dtype: int64


**New function to check below**

In [167]:
def fill_home_planet_by_group(df):
    # Group by 'Group' and then apply a function to fill NA values in 'HomePlanet' within each group
    def fill_planet(group):
        # If there is at least one non-NA 'HomePlanet', use the first one as the fill value
        if group['HomePlanet'].notna().any():
            planet_to_fill = group['HomePlanet'].dropna().iloc[0]
            group['HomePlanet'].fillna(planet_to_fill, inplace=True)
        return group
    
    # Apply the function to each group
    df = df.groupby('Group').apply(fill_planet)
    return df

In [168]:
togeth = fill_home_planet_by_group(togeth)

/var/folders/nk/n25bjxk97j7213hk3kmrl6bm0000gp/T/ipykernel_59632/3070436974.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  group['HomePlanet'].fillna(planet_to_fill, inplace=True)
/var/folders/nk/n25bjxk97j7213hk3kmrl6bm0000gp/T/ipykernel_59632/3070436974.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or exp

In [144]:
togeth[togeth.Group == '0753']

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,Name,Set,Transported,Group,GroupNumber,Deck,Number,Side,FristName,LastName
711,0753_01,Europa,True,A/4/P,NaN,34.0,False,0.0,0.0,0.0,...,Mesarix Ennalve,Train,True,0753,01,A,4,P,Mesarix,Ennalve
712,0753_02,Earth,False,A/4/P,55 Cancri e,65.0,False,19.0,1344.0,4.0,...,NaN,Train,False,0753,02,A,4,P,NaN,NaN
713,0753_03,Europa,True,NaN,55 Cancri e,42.0,False,0.0,0.0,0.0,...,Matan Ennalve,Train,True,0753,03,NaN,NaN,NaN,Matan,Ennalve


In [96]:
checks(df_mini)


Earth
Deck
G    1397
F     938
E     227
A       1
Name: count, dtype: int64

Europa
Deck
B    430
C    416
D    125
A    114
E     80
T      6
F      3
G      1
Name: count, dtype: int64

Mars
Deck
F    723
E    208
D    150
G      3
B      1
Name: count, dtype: int64


In [7]:
def fill_home_planet_by_last_name(df):
    print("Function started")  # Confirm function is called
    planet_to_fill = None  # Initialize the variable outside of the loop
    
    for index, row in df.iterrows():
        print("Checking row:", index)  # Print index to track progress
        if pd.isna(row['HomePlanet']):
            print("HomePlanet is NaN for index:", index)  # Confirm NaN entry found
            matching_rows = df[df['LastName'] == row['LastName']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            
            if not non_nan_planet.empty:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                print("Filling HomePlanet at index", index, "with", planet_to_fill)
                df.at[index, 'HomePlanet'] = planet_to_fill
            else:
                print("No non-NaN HomePlanet found for last name:", row['LastName'])

    return df


## Homeplanet implies deck

In [176]:
togeth.isna().sum()

PassengerId        0
HomePlanet       283
CryoSleep        310
Cabin            299
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name               0
Set                0
Transported     4277
Group              0
GroupNumber        0
Deck             299
Number           299
Side             299
FirstName          0
LastName         294
dtype: int64

In [178]:
togeth = fill_home_planet_by_group(togeth)

In [95]:
def checks(df):
    for home in homeplanets:
        print("\n" + home)
        print(df[df.HomePlanet == home].Deck.value_counts())

People from Earth are only in decks G, F, E
People from Europa are only in decks B,C,A,D,E,T
people from Mars are only in decks F,E,D

In [145]:
def fill_home_planet_by_last_name(df):
    for index,row in df.iterrows():
        if pd.isna(row['HomePlanet']):
            matching_rows = df[df['LastName'] == row['LastName']]
            non_nan_planet = matching_rows.dropna(subset=['HomePlanet'])
            if len(non_nan_planet) > 1:
                planet_to_fill = non_nan_planet['HomePlanet'].values[0]
                df.at[index, 'HomePlanet'] = planet_to_fill
    return df

In [155]:
togeth = home_planet_from_deck_abc(togeth)
togeth = home_planet_from_deck_g(togeth)
togeth = fill_home_planet_by_last_name(togeth)

In [115]:
def home_planet_from_deck_abc(df):
    df.loc[df['Deck'].isin(['A', 'B', 'C']), 'HomePlanet'] = df.loc[df['Deck'].isin(['A', 'B', 'C']), 'HomePlanet'].fillna('Europa')
    return df
def home_planet_from_deck_g(df):
    df.loc[df['Deck']== 'G', 'HomePlanet'] = df.loc[df['Deck'] == 'G', 'HomePlanet'].fillna('Earth')
    return df


In [116]:
def impute(df):
    df = fill_home_planet_by_group(df)
    df = fill_home_planet_by_last_name(df)
    df = home_planet_from_deck_abc(df)
    df = home_planet_from_deck_g(df)
    return df 

togeth = impute(togeth)
togeth = togeth.sort_values(by = ['Group','GroupNumber'])


In [68]:
togeth.isna().sum()

PassengerId        0
HomePlanet         9
CryoSleep        310
Cabin            299
Destination      274
Age              270
VIP              296
RoomService      263
FoodCourt        289
ShoppingMall     306
Spa              284
VRDeck           268
Name               0
Set                0
Transported     4277
Group              0
GroupNumber        0
Deck             299
Number           299
Side             299
FirstName          0
LastName         294
dtype: int64

In [69]:
togeth.isna().sum().sum()

8336

Empty Cabins 299

In [70]:
togeth = togeth[['PassengerId','HomePlanet','Cabin','Destination','Group','GroupNumber','Deck','Number','Side','Set']]

In [99]:
togeth

,PassengerId,HomePlanet,Cabin,Destination,Group,GroupNumber,Deck,Number,Side,Set
0,0001_01,Europa,B/0/P,TRAPPIST-1e,1.0,1.0,B,0.0,P,Train
1,0002_01,Earth,F/0/S,TRAPPIST-1e,2.0,1.0,F,0.0,S,Train
2,0003_01,Europa,A/0/S,TRAPPIST-1e,3.0,1.0,A,0.0,S,Train
3,0003_02,Europa,A/0/S,TRAPPIST-1e,3.0,2.0,A,0.0,S,Train
4,0004_01,Earth,F/1/S,TRAPPIST-1e,4.0,1.0,F,1.0,S,Train
...,...,...,...,...,...,...,...,...,...,...
4276,9277_01,Earth,G/1498/S,PSO J318.5-22,9277.0,1.0,G,1498.0,S,Test
8689,9278_01,Earth,G/1499/S,PSO J318.5-22,9278.0,1.0,G,1499.0,S,Train
8690,9279_01,Earth,G/1500/S,TRAPPIST-1e,9279.0,1.0,G,1500.0,S,Train
8691,9280_01,Europa,E/608/S,55 Cancri e,9280.0,1.0,E,608.0,S,Train


In [72]:
togeth[togeth.Group == 1]

,PassengerId,HomePlanet,Cabin,Destination,Group,GroupNumber,Deck,Number,Side,Set
0,0001_01,Europa,B/0/P,TRAPPIST-1e,1.0,1.0,B,0.0,P,Train


In [73]:
list(togeth[togeth.Deck == 'B'].Cabin.unique())

['B/0/P',
 'B/1/P',
 'B/2/P',
 'B/3/P',
 'B/0/S',
 'B/4/P',
 'B/5/P',
 'B/1/S',
 'B/6/P',
 'B/7/P',
 'B/8/P',
 'B/2/S',
 'B/3/S',
 'B/9/P',
 'B/10/P',
 'B/4/S',
 'B/11/P',
 'B/5/S',
 'B/6/S',
 'B/12/P',
 'B/7/S',
 'B/8/S',
 'B/9/S',
 'B/14/P',
 'B/10/S',
 'B/15/P',
 'B/11/S',
 'B/16/P',
 'B/17/P',
 'B/18/P',
 'B/19/P',
 'B/20/P',
 'B/13/S',
 'B/21/P',
 'B/14/S',
 'B/15/S',
 'B/22/P',
 'B/16/S',
 'B/17/S',
 'B/18/S',
 'B/19/S',
 'B/20/S',
 'B/21/S',
 'B/22/S',
 'B/23/P',
 'B/23/S',
 'B/24/P',
 'B/25/P',
 'B/26/P',
 'B/27/P',
 'B/24/S',
 'B/28/P',
 'B/25/S',
 'B/29/P',
 'B/31/P',
 'B/26/S',
 'B/27/S',
 'B/32/P',
 'B/28/S',
 'B/33/P',
 'B/29/S',
 'B/30/S',
 'B/31/S',
 'B/32/S',
 'B/33/S',
 'B/34/S',
 'B/35/S',
 'B/36/S',
 'B/34/P',
 'B/37/S',
 'B/35/P',
 'B/38/S',
 'B/39/S',
 'B/36/P',
 'B/40/S',
 'B/37/P',
 'B/38/P',
 'B/41/S',
 'B/42/S',
 'B/39/P',
 'B/40/P',
 'B/43/S',
 'B/41/P',
 'B/44/S',
 'B/45/S',
 'B/46/S',
 'B/42/P',
 'B/43/P',
 'B/47/S',
 'B/48/S',
 'B/44/P',
 'B/49/S',
 'B/50/S

In [86]:
used_cabins = {}
for deck in list(togeth.dropna().Deck.unique()):
    cab = {}
    for side in list(togeth.dropna().Side.unique()):
        rooms = list(togeth[(togeth.Deck == deck) & (togeth.Side == side)].dropna().Number.unique())
        rooms.sort()
        cab[side] = rooms
    used_cabins[deck] = cab
    

#### Same cabin means same group

In [97]:
togeth_cleaned = togeth.dropna(subset=['Group','Cabin'])


# Cabin by 'LastName' and aggregate Group into lists
Cabined = togeth_cleaned.groupby('Cabin')['Group'].agg(list).reset_index()

# Function to check if all elements in a list are the same
def is_uniform(Groups):
    return len(set(Groups)) == 1

# Apply the function to identify uniform and mixed Cabins
Cabined['UniformPlanet'] = Cabined['Group'].apply(is_uniform)

# Count how many people are in uniform and mixed planet Cabins
uniform_Cabins = Cabined[Cabined['UniformPlanet'] == True]
mixed_Cabins = Cabined[Cabined['UniformPlanet'] == False]

# Sum up the number of people in uniform and mixed Cabins
people_uniform = uniform_Cabins['Group'].apply(len).sum()
people_mixed = mixed_Cabins['Group'].apply(len).sum()

print(f"Number of people from Cabins where all share the same Group: {people_uniform}")
print(f"Number of people from Cabins where not all share the same Group: {people_mixed}")


Number of people from Cabins where all share the same Group: 12671
Number of people from Cabins where not all share the same Group: 0


#### fill people that must be sharing a room with people in the same group
ie, from their chome planet we know the decks they could be in, if the side and number below and above are one above and one below then they must be sharing with someone from their group. make sure that their group has only one cabin

need to make sure is only one person  with cabin being na in the group otherwise one na might be in another cabin 

In [ ]:
def sharing_room_with_group_member(df):
    potential_decks = 
    for row in df:

## what can be deduced with these cabins now filled ?